In [ ]:
# ! pip install fastparquet

In [2]:
import pandas as pd
import numpy as np
from glob import glob
import gc


In [3]:
pd.options.display.max_rows = None
pd.options.display.max_columns = None


In [4]:
train = pd.read_parquet("../data_phase1/train.parquet")
val = pd.read_parquet("../data_phase1/validation.parquet")


In [5]:
attributes_df = pd.read_parquet("../data_phase1/attributes.parquet")

In [6]:
train = train.merge(attributes_df, on='product_id', how='left')
val = val.merge(attributes_df, on='product_id', how='left')


In [7]:
train.head(2)

,query_id,user_id,session_id,product_id,page_type,previous_page_type,device_category,device_platform,user_tier,user_country,context_type,context_value,product_price,week,week_day,is_click,gender,main_colour,second_colour,season,collection,category_id_l1,category_id_l2,category_id_l3,brand_id,season_year,start_online_date,material_values,attribute_values
0,92d4dd491a874a2cf92c8d311a44a42b597c64a5ede23d...,e5e4c71b1b9456dafece1338762d4ee3db698cf32c384c...,2740b0d77b4e6fafd75321f7d0794210afa8bd650955e7...,bf056e3841dd3a358c6aacb1f9e74e4c7c4adc62e33b45...,596618814963e496d74434df8b8fe3306892f2e4ce6aaa...,c9f34437ce0e536fefd11a34b9a411b541d2dabfec872a...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,fec032cb05435471f2305006f4a1ba994c9d2f4bcad8ef...,designer_id,e5315dbea15a033bc6974a0bccf5fae4a017648bcd92ab...,0.000263,7,6,0,2e7778b3e29aa8df403a1b4fd4c26f848d7a739fd0cb03...,752e245892cce8aa87b8f41c583f95c7059cbaa4e9d61b...,ca8c396e7422e324d4454a911e0319d07b85a4fb89b006...,9db35d237f5873f0410d3ca18c07430270086eb1e7838d...,4c8006c7e513057a9138641abc2a9e65f4b014a8045259...,642878c18e40a1028b6610cd3396d4ee4fb6ad9623f5bb...,a37fdccd28723fba01a2f0a68d3bb1a3b847e550a7b0c2...,ca8c396e7422e324d4454a911e0319d07b85a4fb89b006...,206b88b91a241cc6ed5af856ce3ddda4dccf91ae2ec798...,dc1194ba428d5cd4c49f8a769a6577ac1042162da38bc1...,570.041667,f61ecea9b45f1590e57706b88207449bdd4cb703b917ad...,28d2b1e7b0970e2c58966c47b4c7ab7426e43a8689007e...
1,541a93bd95c3f4127a53e6b0d4b41db55ad9cb9e19d34a...,fca847f7eb5a5a21991421354b0f26afb4a517e540541f...,d5feab37634fd140e85b8f98dcb909a8779b4f0417c73a...,c6513ec49c8e04c265c907933799ff76f24c075c6308c4...,06a7f8e972f61aeb0e06335699518079a444e4450ff766...,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d179859aac8f7c1f88e1ee29b6655596873318c55127d3...,8c82855f15d05cd74fa59956434df17522fc68e4ce3900...,product_id,61205c20046f2688cb7ed03cad29d5a5dbdc360ff48290...,0.012966,6,5,0,51783ef49eeaae84ab72114d1b79f671847df95b30c8e9...,d3034459d822bd39679db3dbc674a30d0d64ba82f24ad4...,0078d3ef1479189e5a922b27e2540a7bb1d10eb9ec0581...,847a067597e39838f1f85b0774f44e68b4d6e64d3ec4dd...,4c8006c7e513057a9138641abc2a9e65f4b014a8045259...,f6bafdefb3c04b392adc1d139ea55a565c8829627bb7d3...,884c59e2814e352d318a8447c2f32a0370955af64838e3...,393b930ad3c40c6af1c14ce8572f39a3a2027ef70dd21e...,de60eee3c057eab83a9987275b3ecfb080026c3297b036...,ef7d8d2e02aec8c328dafde95486f7181c37d07be3d167...,208.041667,c4aebf95aa0044c5230c104faa9ef2b5ccfab836b764d4...,ca8c396e7422e324d4454a911e0319d07b85a4fb89b006...


In [8]:
del attributes_df
gc.collect()

30

In [9]:
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df


In [10]:
train = reduce_mem_usage(train)
val = reduce_mem_usage(val)


Memory usage of dataframe is 802.92 MB
Memory usage after optimization is: 274.46 MB
Decreased by 65.8%
Memory usage of dataframe is 152.04 MB
Memory usage after optimization is: 67.06 MB
Decreased by 55.9%


In [11]:
gc.collect()

22

In [12]:
train.shape, val.shape

((3507990, 29), (687192, 28))

In [13]:
BASE_PATH = "../preprocessed_data/clickout_features/"

In [14]:
merge_paths = glob(BASE_PATH+"*.csv")



In [15]:
merge_paths

['../preprocessed_data/clickout_features/product_wise_last_clickout_days_elapsed.csv',
 '../preprocessed_data/clickout_features/category_id_l2_click_percentage.csv',
 '../preprocessed_data/clickout_features/clicked_product_frequency_by_week.csv',
 '../preprocessed_data/clickout_features/brand_id_click_percentage.csv',
 '../preprocessed_data/clickout_features/product_wise_last_clickout_product_price.csv',
 '../preprocessed_data/clickout_features/clicked_product_prices_by_week.csv',
 '../preprocessed_data/clickout_features/category_id_l3_click_percentage.csv',
 '../preprocessed_data/clickout_features/season_year_click_percentage.csv',
 '../preprocessed_data/clickout_features/weekly_mean_product_start_online_date_clicked_out.csv',
 '../preprocessed_data/clickout_features/weekly_min_product_start_online_date_clicked_out.csv',
 '../preprocessed_data/clickout_features/unique_user_clicks_per_product.csv',
 '../preprocessed_data/clickout_features/weekly_max_product_start_online_date_clicked_ou

In [16]:
len(merge_paths)

15

In [17]:
done_files = []

In [18]:
def merge_data(train_data, val_data, merge_data_path_list, merge_cols='', flag=0):
    for path in merge_data_path_list:
        print("Merging file...", path.split('/')[-1])
        d = pd.read_csv(path)
        d = reduce_mem_usage(d)
        if 'is_click' in d.columns.tolist():
            d = d.drop('is_click', 1)
        if flag==0:
            merge_cols = [d.columns[0]]
        train_data = train_data.merge(d, on=merge_cols, how='left')
        val_data = val_data.merge(d, on=merge_cols, how='left')
        print("Train: ", train_data.shape, "Val: ", val_data.shape)
        done_files.append(path)
        del d
        gc.collect()
    
    return train_data, val_data


In [19]:
train, val = merge_data(train, val, merge_paths[:6])



Merging file... product_wise_last_clickout_days_elapsed.csv
Memory usage of dataframe is 3.19 MB
Memory usage after optimization is: 10.66 MB
Decreased by -233.9%
Train:  (3507990, 30) Val:  (687192, 29)
Merging file... category_id_l2_click_percentage.csv
Memory usage of dataframe is 0.01 MB
Memory usage after optimization is: 0.01 MB
Decreased by -105.5%
Train:  (3507990, 31) Val:  (687192, 30)
Merging file... clicked_product_frequency_by_week.csv
Memory usage of dataframe is 28.03 MB
Memory usage after optimization is: 27.03 MB
Decreased by 3.6%
Train:  (3507990, 39) Val:  (687192, 38)
Merging file... brand_id_click_percentage.csv
Memory usage of dataframe is 0.05 MB
Memory usage after optimization is: 0.16 MB
Decreased by -223.1%
Train:  (3507990, 40) Val:  (687192, 39)
Merging file... product_wise_last_clickout_product_price.csv
Memory usage of dataframe is 3.19 MB
Memory usage after optimization is: 10.85 MB
Decreased by -240.2%
Train:  (3507990, 41) Val:  (687192, 40)
Merging fil

In [20]:
[each.split('/')[-1] for each in done_files]


['product_wise_last_clickout_days_elapsed.csv',
 'category_id_l2_click_percentage.csv',
 'clicked_product_frequency_by_week.csv',
 'brand_id_click_percentage.csv',
 'product_wise_last_clickout_product_price.csv',
 'clicked_product_prices_by_week.csv']

In [21]:
# done_files = [BASE_PATH+each for each in done_files]
# done_files

In [22]:
train.shape, val.shape

((3507990, 49), (687192, 48))

In [23]:
train.memory_usage().sum()/(1024*1024*1024), val.memory_usage().sum()/(1024*1024*1024)

(0.41259583085775375, 0.09557408094406128)

In [24]:
merge_paths = list(set(merge_paths) - set(done_files))
merge_paths

['../preprocessed_data/clickout_features/weekly_mean_product_start_online_date_clicked_out.csv',
 '../preprocessed_data/clickout_features/category_id_l1_click_percentage.csv',
 '../preprocessed_data/clickout_features/second_colour_click_percentage.csv',
 '../preprocessed_data/clickout_features/unique_user_clicks_per_product.csv',
 '../preprocessed_data/clickout_features/season_year_click_percentage.csv',
 '../preprocessed_data/clickout_features/weekly_min_product_start_online_date_clicked_out.csv',
 '../preprocessed_data/clickout_features/main_colour_click_percentage.csv',
 '../preprocessed_data/clickout_features/weekly_max_product_start_online_date_clicked_out.csv',
 '../preprocessed_data/clickout_features/category_id_l3_click_percentage.csv']

In [25]:
train, val = merge_data(train, val, merge_paths[:6])


Merging file... weekly_mean_product_start_online_date_clicked_out.csv
Memory usage of dataframe is 28.03 MB
Memory usage after optimization is: 27.03 MB
Decreased by 3.6%
Train:  (3507990, 57) Val:  (687192, 56)
Merging file... category_id_l1_click_percentage.csv
Memory usage of dataframe is 0.00 MB
Memory usage after optimization is: 0.00 MB
Decreased by -105.0%
Train:  (3507990, 58) Val:  (687192, 57)
Merging file... second_colour_click_percentage.csv
Memory usage of dataframe is 0.00 MB
Memory usage after optimization is: 0.00 MB
Decreased by -92.9%
Train:  (3507990, 59) Val:  (687192, 58)
Merging file... unique_user_clicks_per_product.csv
Memory usage of dataframe is 6.23 MB
Memory usage after optimization is: 21.58 MB
Decreased by -246.3%
Train:  (3507990, 60) Val:  (687192, 59)
Merging file... season_year_click_percentage.csv
Memory usage of dataframe is 0.00 MB
Memory usage after optimization is: 0.00 MB
Decreased by -101.8%
Train:  (3507990, 61) Val:  (687192, 60)
Merging file.

In [26]:
merge_paths = list(set(merge_paths) - set(done_files))
merge_paths

['../preprocessed_data/clickout_features/main_colour_click_percentage.csv',
 '../preprocessed_data/clickout_features/weekly_max_product_start_online_date_clicked_out.csv',
 '../preprocessed_data/clickout_features/category_id_l3_click_percentage.csv']

In [27]:
train, val = merge_data(train, val, merge_paths)


Merging file... main_colour_click_percentage.csv
Memory usage of dataframe is 0.00 MB
Memory usage after optimization is: 0.00 MB
Decreased by -117.8%
Train:  (3507990, 70) Val:  (687192, 69)
Merging file... weekly_max_product_start_online_date_clicked_out.csv
Memory usage of dataframe is 28.03 MB
Memory usage after optimization is: 27.03 MB
Decreased by 3.6%
Train:  (3507990, 78) Val:  (687192, 77)
Merging file... category_id_l3_click_percentage.csv
Memory usage of dataframe is 0.01 MB
Memory usage after optimization is: 0.02 MB
Decreased by -151.9%
Train:  (3507990, 79) Val:  (687192, 78)


In [28]:
train.head(2)

,query_id,user_id,session_id,product_id,page_type,previous_page_type,device_category,device_platform,user_tier,user_country,context_type,context_value,product_price,week,week_day,is_click,gender,main_colour,second_colour,season,collection,category_id_l1,category_id_l2,category_id_l3,brand_id,season_year,start_online_date,material_values,attribute_values,last_clickout_days_elapsed,percentage_category_id_l2_click,click_frequency_week_1,click_frequency_week_2,click_frequency_week_3,click_frequency_week_4,click_frequency_week_5,click_frequency_week_6,click_frequency_week_7,click_frequency_week_8,percentage_brand_id_click,last_clickout_product_price,clickout_product_mean_price_week_1,clickout_product_mean_price_week_2,clickout_product_mean_price_week_3,clickout_product_mean_price_week_4,clickout_product_mean_price_week_5,clickout_product_mean_price_week_6,clickout_product_mean_price_week_7,clickout_product_mean_price_week_8,start_online_date_mean_clicked_out_week_1,start_online_date_mean_clicked_out_week_2,start_online_date_mean_clicked_out_week_3,start_online_date_mean_clicked_out_week_4,start_online_date_mean_clicked_out_week_5,start_online_date_mean_clicked_out_week_6,start_online_date_mean_clicked_out_week_7,start_online_date_mean_clicked_out_week_8,percentage_category_id_l1_click,percentage_second_colour_click,#unique_users_clicked,percentage_season_year_click,start_online_date_min_clicked_out_week_1,start_online_date_min_clicked_out_week_2,start_online_date_min_clicked_out_week_3,start_online_date_min_clicked_out_week_4,start_online_date_min_clicked_out_week_5,start_online_date_min_clicked_out_week_6,start_online_date_min_clicked_out_week_7,start_online_date_min_clicked_out_week_8,percentage_main_colour_click,start_online_date_max_clicked_out_week_1,start_online_date_max_clicked_out_week_2,start_online_date_max_clicked_out_week_3,start_online_date_max_clicked_out_week_4,start_online_date_max_clicked_out_week_5,start_online_date_max_clicked_out_week_6,start_online_date_max_clicked_out_week_7,start_online_date_max_clicked_out_week_8,percentage_category_id_l3_click
0,92d4dd491a874a2cf92c8d311a44a42b597c64a5ede23d...,e5e4c71b1b9456dafece1338762d4ee3db698cf32c384c...,2740b0d77b4e6fafd75321f7d0794210afa8bd650955e7...,bf056e3841dd3a358c6aacb1f9e74e4c7c4adc62e33b45...,596618814963e496d74434df8b8fe3306892f2e4ce6aaa...,c9f34437ce0e536fefd11a34b9a411b541d2dabfec872a...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,fec032cb05435471f2305006f4a1ba994c9d2f4bcad8ef...,designer_id,e5315dbea15a033bc6974a0bccf5fae4a017648bcd92ab...,0.000263,7,6,0,2e7778b3e29aa8df403a1b4fd4c26f848d7a739fd0cb03...,752e245892cce8aa87b8f41c583f95c7059cbaa4e9d61b...,ca8c396e7422e324d4454a911e0319d07b85a4fb89b006...,9db35d237f5873f0410d3ca18c07430270086eb1e7838d...,4c8006c7e513057a9138641abc2a9e65f4b014a8045259...,642878c18e40a1028b6610cd3396d4ee4fb6ad9623f5bb...,a37fdccd28723fba01a2f0a68d3bb1a3b847e550a7b0c2...,ca8c396e7422e324d4454a911e0319d07b85a4fb89b006...,206b88b91a241cc6ed5af856ce3ddda4dccf91ae2ec798...,dc1194ba428d5cd4c49f8a769a6577ac1042162da38bc1...,570.0,f61ecea9b45f1590e57706b88207449bdd4cb703b917ad...,28d2b1e7b0970e2c58966c47b4c7ab7426e43a8689007e...,33.0,0.373047,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.027390,0.000260,0.0,0.000000,0.000000,0.000000,0.000260,0.000000,0.000263,0.000000,NaN,NaN,NaN,NaN,570.0,NaN,570.0,NaN,4.292969,84.437500,1,18.640625,NaN,NaN,NaN,NaN,570.0,NaN,570.0,NaN,6.007812,NaN,NaN,NaN,NaN,570.0,NaN,570.0,NaN,50.156250
1,541a93bd95c3f4127a53e6b0d4b41db55ad9cb9e19d34a...,fca847f7eb5a5a21991421354b0f26afb4a517e540541f...,d5feab37634fd140e85b8f98dcb909a8779b4f0417c73a...,c6513ec49c8e04c265c907933799ff76f24c075c6308c4...,06a7f8e972f61aeb0e06335699518079a444e4450ff766...,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d179859aac8f7c1f88e1ee29

In [29]:
cols_to_drop = [
 'page_type', 'previous_page_type', 'device_category', 'device_platform', 'user_tier',
 'user_country', 'context_type', 'context_value', 'product_price', 'week', 'week_day',
 'gender', 'main_colour', 'second_colour', 'season', 'collection', 'category_id_l1', 'category_id_l2',
 'category_id_l3', 'brand_id', 'season_year', 'start_online_date', 'material_values', 'attribute_values'
]


In [30]:
train = train.drop(columns=cols_to_drop+['is_click'], axis=1)
val = val.drop(columns=cols_to_drop, axis=1)


In [31]:
train.head()

,query_id,user_id,session_id,product_id,last_clickout_days_elapsed,percentage_category_id_l2_click,click_frequency_week_1,click_frequency_week_2,click_frequency_week_3,click_frequency_week_4,click_frequency_week_5,click_frequency_week_6,click_frequency_week_7,click_frequency_week_8,percentage_brand_id_click,last_clickout_product_price,clickout_product_mean_price_week_1,clickout_product_mean_price_week_2,clickout_product_mean_price_week_3,clickout_product_mean_price_week_4,clickout_product_mean_price_week_5,clickout_product_mean_price_week_6,clickout_product_mean_price_week_7,clickout_product_mean_price_week_8,start_online_date_mean_clicked_out_week_1,start_online_date_mean_clicked_out_week_2,start_online_date_mean_clicked_out_week_3,start_online_date_mean_clicked_out_week_4,start_online_date_mean_clicked_out_week_5,start_online_date_mean_clicked_out_week_6,start_online_date_mean_clicked_out_week_7,start_online_date_mean_clicked_out_week_8,percentage_category_id_l1_click,percentage_second_colour_click,#unique_users_clicked,percentage_season_year_click,start_online_date_min_clicked_out_week_1,start_online_date_min_clicked_out_week_2,start_online_date_min_clicked_out_week_3,start_online_date_min_clicked_out_week_4,start_online_date_min_clicked_out_week_5,start_online_date_min_clicked_out_week_6,start_online_date_min_clicked_out_week_7,start_online_date_min_clicked_out_week_8,percentage_main_colour_click,start_online_date_max_clicked_out_week_1,start_online_date_max_clicked_out_week_2,start_online_date_max_clicked_out_week_3,start_online_date_max_clicked_out_week_4,start_online_date_max_clicked_out_week_5,start_online_date_max_clicked_out_week_6,start_online_date_max_clicked_out_week_7,start_online_date_max_clicked_out_week_8,percentage_category_id_l3_click
0,92d4dd491a874a2cf92c8d311a44a42b597c64a5ede23d...,e5e4c71b1b9456dafece1338762d4ee3db698cf32c384c...,2740b0d77b4e6fafd75321f7d0794210afa8bd650955e7...,bf056e3841dd3a358c6aacb1f9e74e4c7c4adc62e33b45...,33.0,0.373047,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.027390,0.000260,0.0,0.000000,0.000000,0.000000,0.000260,0.000000,0.000263,0.000000,NaN,NaN,NaN,NaN,570.0,NaN,570.0,NaN,4.292969,84.437500,1,18.640625,NaN,NaN,NaN,NaN,570.0,NaN,570.0,NaN,6.007812,NaN,NaN,NaN,NaN,570.0,NaN,570.0,NaN,50.156250
1,541a93bd95c3f4127a53e6b0d4b41db55ad9cb9e19d34a...,fca847f7eb5a5a21991421354b0f26afb4a517e540541f...,d5feab37634fd140e85b8f98dcb909a8779b4f0417c73a...,c6513ec49c8e04c265c907933799ff76f24c075c6308c4...,47.0,0.875000,0.0,1.0,2.0,2.0,5.0,9.0,5.0,0.0,0.604492,0.012932,0.0,0.013206,0.013206,0.013214,0.013542,0.013458,0.012939,0.013443,NaN,208.0,208.0,208.0,208.0,208.0,208.0,208.0,1.053711,0.791016,23,69.125000,NaN,208.0,208.0,208.0,208.0,208.0,208.0,208.0,4.250000,NaN,208.0,208.0,208.0,208.0,208.0,208.0,208.0,0.331543
2,263ea1e38126fe0c7bfbff24a33b1a09f4dac4f8cd4bb4...,90ea15d8d96a9d3e7ab463d990e5f4565cac9477498d37...,18ae37a1a05faa7fd54818794a1a8e44073e00a56fa05f...,b4d5e28da10318aa7776b364528dc92f83ba45326018b5...,42.0,3.068359,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,3.027344,0.002733,0.0,0.002720,0.002714,0.002697,0.002703,0.002733,0.002733,0.002739,NaN,549.0,549.0,549.0,549.0,549.0,549.0,549.0,11.867188,84.437500,4,18.640625,NaN,549.0,549.0,549.0,549.0,549.0,549.0,549.0,6.007812,NaN,549.0,549.0,549.0,549.0,549.0,549.0,549.0,50.156250
3,3727580d84ce2fbe42ff8bc6f732331f65ea659864a04c...,8f88d89f2a71e2adf42f885fa6adedd09bf039843b535c...,10667a5a6047aa173d13997cdcd996cbdaf9b0149f9655...,5a36f600d3c01763c28e2dafc53119fba7bcc6a867ab8f...,36.0,2.080078,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.005985,0.000070,0.0,0.000000,0.000000,0.000000,0.000070,0.000070,0.000000,0.000000,NaN,NaN,NaN,NaN,581.0,581.0,NaN,NaN,14.062500,84.437500,1,18.640625,NaN,NaN,NaN,NaN,581.0,581.0,NaN,NaN,10.359375,NaN,NaN,NaN,NaN,581.0,581.0,NaN,NaN,1.968750
4,1fcf5d263785455311cecf2f864eaa2eeca4da488383d9...,6b21688d90dfd9677fa7979dfc6da5b963c0e1e3d68a38...,8b39f00bfc9d45ed6f64dd39a72548936a157f9f03e7f4...,314d20e9e9ec3e97d1867c8bf8c6feb0c23d918

In [36]:
train.shape

(3507990, 54)

In [35]:
train.isnull().sum()

query_id                                           0
user_id                                            0
session_id                                         0
product_id                                         0
last_clickout_days_elapsed                    569952
percentage_category_id_l2_click                    0
click_frequency_week_1                             0
click_frequency_week_2                             0
click_frequency_week_3                             0
click_frequency_week_4                             0
click_frequency_week_5                             0
click_frequency_week_6                             0
click_frequency_week_7                             0
click_frequency_week_8                             0
percentage_brand_id_click                          0
last_clickout_product_price                   569952
clickout_product_mean_price_week_1                 0
clickout_product_mean_price_week_2                 0
clickout_product_mean_price_week_3            